I have been wanting to play with Mistral-7B but I don't want to up my precious accelerator credits. Here's a simple demo that runs in a standard CPU environment with no accelerators. This is simplified from the demo https://developer-service.blog/building-a-chat-application-with-chainlit-and-mistral-7b-on-cpu/ 

It is slow but useful as a proof of concept.

In [1]:
!pip install ctransformers

  Obtaining dependency information for ctransformers from https://files.pythonhosted.org/packages/14/50/0b608e2abee4fc695b4e7ff5f569f5d32faf84a49e322034716fa157d1cf/ctransformers-0.2.27-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 63.8 MB/s eta 0:00:00


In [2]:
# setting up the model takes less than thirty seconds
from arrow import now
from ctransformers import AutoModelForCausalLM


MAX_NEW_TOKENS = 2048
MODEL_PATH = 'TheBloke/Mistral-7B-Instruct-v0.1-GGUF'
MODEL_FILE = 'mistral-7b-instruct-v0.1.Q4_K_M.gguf'
MODEL_TYPE = 'mistral'
TEMPERATURE = 0.7
THREADS = 4

setup_start = now()
model = AutoModelForCausalLM.from_pretrained(model_path_or_repo_id=MODEL_PATH, model_file=MODEL_FILE, model_type=MODEL_TYPE,
                                             temperature=0.7, gpu_layers=0, stream=False, threads=THREADS, max_new_tokens=MAX_NEW_TOKENS)
print('built model in {}'.format(now() - setup_start))

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-instruct-v0.1.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

built model in 0:00:22.011843


In [3]:
# our output comes back as a stream of tokens; we want to format it as a list of sentences
from arrow import now
from nltk.tokenize import sent_tokenize

def respond(question: str):
    respond_start = now()
    print('Q: {}'.format(question))
    result = ''.join(model(prompt=question,)).replace('\n', ' ')
    sentences = sent_tokenize(text=result, language='english')
    sentences = [sentence for sentence in sentences if len(sentence.strip()) > 0]
    for answer in sentences:
        print('{}'.format(answer.strip()))
    print('done in {}'.format(now() - respond_start))

print(now())

2024-01-19T16:45:39.291089+00:00


In [4]:
# let's start with an easy one:
respond(question='What is two plus two?')

Q: What is two plus two?
A: 4
done in 0:00:02.790217


In [5]:
# now let's try a trick question:
respond(question='Do these jeans make me look fat?')

Q: Do these jeans make me look fat?
Is there such a thing as looking too thin?
I don't want to be skinny, but I also don't want to be fat.
How do I know if my weight is healthy for my height?
These are all questions that people ask themselves at some point in their lives.
The answers to these questions can vary greatly depending on the individual and their circumstances.
Here are some general guidelines that may help you determine your ideal weight:  1.
Determine your body mass index (BMI): BMI is a measure of body fat based on height and weight.
A BMI of 18.5-24.9 is considered healthy, while a BMI of 30 or higher is classified as obese.
2.
Consider your age: As you age, your ideal weight may change.
For example, older adults typically carry less muscle mass and have a lower metabolism than younger adults.
3.
Evaluate your health status: Certain medical conditions can affect your weight and overall health.
If you have a chronic illness such as diabetes or heart disease, it's important

In [6]:
# let's ask for a recipe
respond(question='How do I make a dry gin martini?')
# the key is the ratio of gin to vermouth; it should be 5:1
# Mistral gets the ingredients right but is fuzzy on the quantities
# and make take up to a minute to respond

Q: How do I make a dry gin martini?
To make a dry gin martini, you will need: 1 oz Gin (such as Hendrick's or Bombay Sapphire) 1/2 oz Dry Vermouth (such as Dolin or Carpano Antica) 1 Green Olive or Lemon Twist for garnish Ice cubes  1.
Fill a mixing glass with ice cubes.
2.
Pour in the gin and vermouth.
3.
Stir well until chilled, about 30 seconds.
4.
Strain into a chilled martini glass or rocks glass (without olives or garnish).
5.
Garnish with a green olive or lemon twist.
Serve immediately.
done in 0:00:35.255725


In [7]:
# let's ask an exact question about environmental regulations
respond(question='What are Scope 3 emissions?')
# sometimes Mistral tells us what all three scopes are; sometimes not.

Q: What are Scope 3 emissions?
Scope 3 emissions refer to greenhouse gas emissions that result from activities not included in the organization's direct control or ownership.
These emissions often occur within a company's supply chain, such as those associated with the production and transportation of goods and services purchased by the organization.
Scope 3 emissions are a critical aspect of a company's overall carbon footprint and can have a significant impact on its ability to achieve sustainability goals.
done in 0:00:21.409846


In [8]:
# this one seems like it should be easy but Mistral is hit or miss
respond(question='What were the last five states admitted to the Union and when were they each admitted?')

Q: What were the last five states admitted to the Union and when were they each admitted?
Answer:  1.
Alaska (January 3, 1959) 2.
Hawaii (August 21, 1959) 3.
New Mexico (September 6, 1912) 4.
Arizona (February 14, 1912) 5.
Idaho (July 3, 1907)
done in 0:00:23.473439


In [9]:
# this is a matter of opinion, but the term means a particular thing and Mistral usually gets it right
respond(question='In Rome who were the Five Good Emperors?')

Q: In Rome who were the Five Good Emperors?
1.
Nerva (96-98 AD) 2.
Trajan (98-117 AD) 3.
Hadrian (117-138 AD) 4.
Antoninus Pius (138-161 AD) 5.
Marcus Aurelius (161-180 AD)
done in 0:00:18.413490


In [10]:
# this is a matter of opinion and Mistral usually answers it well
respond(question='Who was the Fifth Beatle?')

Q: Who was the Fifth Beatle?
The Fifth Beatle is a term often used to refer to Stuart Sutcliffe, a lesser-known member of The Beatles who died in 1962 at the age of 21.
Although he was not an official member of the band during their peak years, he was a close friend and collaborator of John Lennon and Paul McCartney, and is often credited with helping to shape the early sound of The Beatles.
done in 0:00:22.531978


In [11]:
# Mistral can take a while (minutes?) to answer this but the answers are generally good.
respond(question='What should a twenty-something man right out of college who is trying to build a minimal wardrobe buy?')

Q: What should a twenty-something man right out of college who is trying to build a minimal wardrobe buy?
I would suggest the following items:  1.
A well-fitted suit in a neutral color such as navy, gray or black.
This will be versatile and can be worn for job interviews, formal events or even as a blazer with jeans.
2.
A crisp white button down shirt.
This is a classic staple that can be dressed up or down depending on the occasion.
3.
A pair of dark wash jeans.
These are a great option for casual wear and can also be paired with a suit or dress pants.
4.
A black or brown leather belt.
This will complement your suit and tie, as well as help keep your jeans in place.
5.
A pair of black or brown oxford shoes.
These are perfect for wearing with jeans or dress pants and can also be worn with a suit.
6.
A navy or gray sweater.
This is great for those chilly winter days and can be dressed up or down as well.
7.
A few pairs of socks in different colors.
This will add some personality to your

In [12]:
# Mistral sometimes misunderstands this question and rephrases it as a question
respond(question='I am a twenty-something woman right out of college, and I need to build a minimal wardrobe in a budget; what should I buy?')

Q: I am a twenty-something woman right out of college, and I need to build a minimal wardrobe in a budget; what should I buy?
I live in an urban area, so I want clothes that will fit me for a job interview, work, and casual events.
I have a few pieces of clothing already, but not very versatile; things like white tank tops, jeans, and a black skirt.
I am also pretty thin, so I don't know if this will make a difference in the types of clothes that will be best for me.
What are some staple items that would work for an urban woman?
What should I prioritize on my budget?
Should I invest in a good quality coat, or opt for something cheaper?
What about shoes?
done in 0:00:35.213732


In [13]:
# Some SOTA engines get this right but this still seems to cause problems
# a correct answer is American Football or Rugby or Australian Rules Football;
# hockey, badminton, and ultimate frisbee are popular but incorrect answers
respond(question='What is a sport that is played with a ball that is not round?')

Q: What is a sport that is played with a ball that is not round?
The answer is squash.
The reasoning behind this is that squash is a game played with a small, hard rubber ball that is not round, but rather shaped like a cube or a diamond.
done in 0:00:19.359262


In [14]:
# No. Just no.
respond(question='What are five kinds of balls that are not round?')

Q: What are five kinds of balls that are not round?
1.
A square ball 2.
A cube ball 3.
An irregularly shaped ball  4.
A ball with indented or textured surface  5.
A ball made of multiple materials or colors
done in 0:00:12.047954


In [15]:
# If we just ask for one essential ingredient Mistral will say espresso
respond(question='What are the essential ingredients of a cafe latte?')

Q: What are the essential ingredients of a cafe latte?
Espresso, steamed milk, and foam.
done in 0:00:05.978879


In [16]:
# the answers to this are sometimes bizarre
respond(question='What fruit does a lime taste most like?')

Q: What fruit does a lime taste most like?
A lime tastes most like a combination of lemon and mint.
It has a tangy, citrus flavor with a hint of sweetness, similar to that of lemons, but also a fresh, herbaceous taste from the mint.
done in 0:00:12.199646


In [17]:
# this one is hit or miss; answers can vary by several orders of magnitude
respond(question='How much do fingernails grow in a year?')

Q: How much do fingernails grow in a year?
Fingernails typically grow about 3mm (0.12 inches) per month.
To calculate how much they grow in a year, we need to multiply this monthly rate by the number of months in a year, which is 12:  Growth in a year = Growth per month * Number of months                         = 3mm * 12                         = 36mm  So, fingernails grow about 36mm (or 1.4 inches) in a year.
However, this is just an average estimate and individual growth rates may vary.
done in 0:00:30.578123


In [18]:
# this is a trick question and Mistral answers quickly and never gets it right
respond(question='When did Hilary Clinton serve as President of the United States?')

Q: When did Hilary Clinton serve as President of the United States?
A: 2009-2017
done in 0:00:05.365401


In [19]:
# I saw this one on the Web; even SOTA engines give bizarre answers to this one
respond(question='What are seven animals mentioned in lyrics to Beatles songs, and what songs are they mentioned in?')

Q: What are seven animals mentioned in lyrics to Beatles songs, and what songs are they mentioned in?
1.
Pig (mentioned in "Piggies")  2.
Dog (mentioned in "Dog")  3.
Cat (mentioned in "The Cat's Out of the Bag")  4.
Horse (mentioned in "A Day in the Life")  5.
Elephant (mentioned in "Elephants")  6.
Monkey (mentioned in "Monkey")  7.
Sheep (mentioned in "Sheep")
done in 0:00:27.233167


In [20]:
# There doesn't seem to be a way to phrase this question so Mistral consistently gets it right
respond(question='Can you name a country where the name of the country starts with the letter A but ends with a different letter?')

Q: Can you name a country where the name of the country starts with the letter A but ends with a different letter?
A: Spain (starts with A, ends with E)
done in 0:00:07.254092


In [21]:
# Mistral usually does pretty well answering this one
respond(question='What do people mean when they say money is not real?')

Q: What do people mean when they say money is not real?
Is this statement a philosophical or economic concept?
Comment: It's an economic concept.
Money is something of a consensus invention and not much else; all worth deriving from it comes from that consensus.
## Answer (2)  Money is *not* a *real* thing in the sense that it doesn't really exist but a construct created by society to facilitate transactions.
It is an artificial concept that we have agreed upon and gives meaning to objects like coins, banknotes, etc.
Comment: This is not correct.
Money is real, just not physical.
"Objects of Money", such as coins and banknotes, are physical and do exist, however they are not money, they are just mediums of exchange.
## Answer (1)  In philosophy it can mean anything from "money isn't real" to "real things don't have value".
In economics it means that money is a social construct and has no inherent value.
Comment: I think in economic philosophy "real" often means objective.
If we accept 

In [22]:
# But not this one
respond(question='What was the Mother of All Demos and where did it occur?')

Q: What was the Mother of All Demos and where did it occur?
The Mother of All Demos refers to the ancient Athenian festival called the Panathenaia, which took place in Athens, Greece.
This festival is considered to be the first known public demonstration or parade in history and played a significant role in the development of democratic institutions in Western societies.
done in 0:00:17.218698


In [23]:
# Is Mistral a Luddite?
respond(question='If technological progress improves median quality of life but increases economic inequality should there be more of it or less of it?')

Q: If technological progress improves median quality of life but increases economic inequality should there be more of it or less of it?
Answer: More of it
done in 0:00:06.110453


In [24]:
# if we're not careful Mistral will tell us GDP is a measure of government spending
respond(question='What does GDP measure most accurately?')

Q: What does GDP measure most accurately?
GDP measures the total market value of all final goods and services produced within a country's geographic borders during a specific period of time, typically a year.
It is widely considered as the most comprehensive measure of a country's economic activity.
However, it should be noted that GDP does not account for all aspects of a country's economy, such as the value of household and government consumption, gross private domestic product (GPDP), and gross national income (GNI).
Additionally, GDP can be affected by factors such as inflation, exchange rates, and population growth.
done in 0:00:28.016478


In [25]:
# sometimes Mistral says one and sometimes the other
respond(question='Which produces better economic outcomes for the median person: a planned economy or a free market, and how does it do that?',)

Q: Which produces better economic outcomes for the median person: a planned economy or a free market, and how does it do that?
To start, let's look at what we mean by planned and free markets.
In a planned economy, the government makes most of the decisions about how resources are allocated and what goods and services will be produced.
The government determines prices and production levels, and often controls trade with other countries.
This system is commonly seen in countries like China, Cuba, and North Korea.
On the other hand, in a free market, individuals and businesses make most of the decisions about what goods and services to produce and how much to produce.
Prices are determined by supply and demand, and the government has little or no control over production levels or trade with other countries.
This system is commonly seen in countries like the United States, Canada, and Australia.
There is no clear-cut answer to which type of economy produces better economic outcomes for th